# ⚡ Power-CLIP vNext: Attribute-Level Zero-Shot

**核心轉向**: Power → Attributes → Device Reasoning

> *"We reformulate NILM zero-shot as an attribute grounding problem rather than direct class matching."*

**架構**:
- 20 Attribute Prototypes (5 tiers)
- Multi-positive Contrastive Loss
- Rule-based Device Reasoning
- LLM Explanation (post-hoc)

**版本**: 2025-12-31 vNext

In [ ]:
# @title 📂 Setup
from google.colab import drive, userdata
import os, json, torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import random

drive.mount('/content/drive')
BASE_PATH = '/content/drive/MyDrive/Reserach/REDD_Dataset'
os.chdir(BASE_PATH)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🔥 Device: {DEVICE}')

EPOCHS = 20
LR = 1e-3
BATCH_SIZE = 32
TEMPERATURE = 0.07

In [ ]:
!pip install -q sentence-transformers google-generativeai

In [ ]:
# @title 📋 Attribute Ontology (20 prototypes)

ATTRIBUTES = {
    # Power Shape (5)
    'PS1': {'name': 'constant_high', 'text': 'sustained high power consumption above 500W'},
    'PS2': {'name': 'constant_low', 'text': 'sustained low power consumption under 200W'},
    'PS3': {'name': 'cyclic', 'text': 'power cycles on and off periodically'},
    'PS4': {'name': 'burst', 'text': 'short bursts of high power'},
    'PS5': {'name': 'fluctuating', 'text': 'continuously fluctuating power levels'},
    # Temporal (4)
    'T1': {'name': 'long_operation', 'text': 'operates continuously for 30+ minutes'},
    'T2': {'name': 'short_usage', 'text': 'used for less than 10 minutes at a time'},
    'T3': {'name': 'periodic_cycle', 'text': 'regular on-off cycles every 15-30 minutes'},
    'T4': {'name': 'instantaneous', 'text': 'very brief operation under 5 minutes'},
    # Load Type (5)
    'L1': {'name': 'resistive', 'text': 'purely resistive heating load with stable power'},
    'L2': {'name': 'inductive_motor', 'text': 'inductive motor load with startup surge'},
    'L3': {'name': 'switching', 'text': 'switching power supply with high frequency noise'},
    'L4': {'name': 'heating_element', 'text': 'resistive heating element with thermal cycling'},
    'L5': {'name': 'compressor', 'text': 'compressor motor with periodic start-stop'},
    # Power Level (4)
    'P1': {'name': 'very_high', 'text': 'power consumption above 1500W'},
    'P2': {'name': 'high', 'text': 'power consumption between 500W and 1500W'},
    'P3': {'name': 'medium', 'text': 'power consumption between 100W and 500W'},
    'P4': {'name': 'low', 'text': 'power consumption below 100W'},
    # Stability (2)
    'S1': {'name': 'stable', 'text': 'very stable and consistent power draw'},
    'S2': {'name': 'variable', 'text': 'highly variable and unpredictable power'},
}

ATTR_IDS = list(ATTRIBUTES.keys())
ATTR_TEXTS = [ATTRIBUTES[k]['text'] for k in ATTR_IDS]

# Device → Attribute Mapping
DEVICE_ATTRS = {
    'fridge': ['L5', 'T3', 'P4', 'PS3', 'S1'],
    'microwave': ['L1', 'T4', 'P1', 'PS4', 'S1'],
    'dish washer': ['L2', 'L4', 'T1', 'P2', 'PS5'],
    'washer dryer': ['L2', 'L4', 'T1', 'P1', 'PS5', 'S2'],
    'electric stove': ['L1', 'T2', 'P1', 'PS1', 'S1'],
    'electric space heater': ['L4', 'T1', 'P2', 'PS1', 'S1'],
}

ALL_DEVICES = list(DEVICE_ATTRS.keys())

print(f'📋 Attributes: {len(ATTR_IDS)}')
print(f'📋 Devices: {len(ALL_DEVICES)}')
for d, attrs in DEVICE_ATTRS.items():
    print(f'   {d}: {attrs}')

In [ ]:
# @title ⚡ Model Definition
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

class HTFCNNEncoder(nn.Module):
    def __init__(self, embed_dim=384):
        super().__init__()
        self.time_branch = nn.Sequential(
            nn.Conv1d(1, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 5, padding=2), nn.BatchNorm1d(128), nn.ReLU(), nn.AdaptiveAvgPool1d(1)
        )
        self.freq_branch = nn.Sequential(
            nn.Conv1d(1, 64, 3, padding=1), nn.BatchNorm1d(64), nn.ReLU(), nn.AdaptiveAvgPool1d(1)
        )
        self.fusion = nn.Sequential(
            nn.Linear(128 + 64 + 6, 256), nn.LayerNorm(256), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(256, embed_dim)
        )
    
    def forward(self, signal, stats):
        mean, std = signal.mean(1, keepdim=True), signal.std(1, keepdim=True) + 1e-6
        norm = (signal - mean) / std
        time_f = self.time_branch(norm.unsqueeze(1)).squeeze(-1)
        fft = torch.abs(torch.fft.rfft(norm, dim=1))
        fft = fft / (fft.max(1, keepdim=True)[0] + 1e-6)
        freq_f = self.freq_branch(fft.unsqueeze(1)).squeeze(-1)
        return self.fusion(torch.cat([time_f, freq_f, stats], dim=1))

def compute_stats(s):
    return np.array([np.mean(s), np.std(s), np.max(s)-np.min(s), np.max(s), np.min(s),
                     np.max(np.abs(np.diff(s))) if len(s)>1 else 0], dtype=np.float32)

print(f'✅ Model: {sum(p.numel() for p in HTFCNNEncoder().parameters()):,} params')

In [ ]:
# @title 📊 Load Data with Attribute Labels
from torch.utils.data import Dataset, DataLoader

class AttributeDataset(Dataset):
    def __init__(self, labels_file, ws=60, exclude_device=None):
        self.samples = []
        with open(labels_file) as f:
            lines = f.readlines()
        
        for i in tqdm(range(0, len(lines)-ws, ws), desc='Loading'):
            chunk = [json.loads(lines[j]) for j in range(i, i+ws)]
            act = set(chunk[ws//2]['active_appliances']) & set(ALL_DEVICES)
            if not act: continue
            
            # Skip if contains excluded device (LOO)
            if exclude_device and exclude_device in act:
                continue
            
            # Collect all attributes from active devices
            attr_set = set()
            for d in act:
                attr_set.update(DEVICE_ATTRS.get(d, []))
            
            if not attr_set: continue
            
            signal = np.array([c['aggregate_power'] for c in chunk], dtype=np.float32)
            
            # Create attribute mask (multi-positive)
            attr_mask = np.zeros(len(ATTR_IDS), dtype=np.float32)
            for attr in attr_set:
                attr_mask[ATTR_IDS.index(attr)] = 1.0
            
            self.samples.append({
                'signal': signal,
                'stats': compute_stats(signal),
                'attr_mask': attr_mask,
                'devices': list(act)
            })
        
        print(f'📊 Samples: {len(self.samples)}')
    
    def __len__(self): return len(self.samples)
    def __getitem__(self, i):
        s = self.samples[i]
        return (torch.tensor(s['signal']), torch.tensor(s['stats']), 
                torch.tensor(s['attr_mask']))

# Load full training data
train_ds = AttributeDataset('combination_labels.jsonl')

In [ ]:
# @title 🔥 Multi-Positive Contrastive Training
import torch.optim as optim

def train_attribute_model(train_ds, epochs=EPOCHS):
    print('=' * 60)
    print('🔥 Attribute-Level Training (Multi-Positive Contrastive)')
    print('=' * 60)
    
    model = HTFCNNEncoder().to(DEVICE)
    text_model = SentenceTransformer('all-MiniLM-L6-v2').to(DEVICE)
    text_model.eval()
    
    # Encode attribute prototypes
    with torch.no_grad():
        attr_embs = text_model.encode(ATTR_TEXTS, convert_to_tensor=True, device=DEVICE)
        attr_embs = F.normalize(attr_embs, p=2, dim=1)  # [K, 384]
    
    optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)
    
    def collate(batch):
        signals = torch.stack([b[0] for b in batch])
        stats = torch.stack([b[1] for b in batch])
        masks = torch.stack([b[2] for b in batch])
        return signals, stats, masks
    
    loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, 
                        drop_last=True, collate_fn=collate)
    
    history = []
    
    for ep in range(epochs):
        model.train()
        total_loss = 0
        
        for signals, stats, masks in tqdm(loader, desc=f'Ep {ep+1}/{epochs}', leave=False):
            signals, stats, masks = signals.to(DEVICE), stats.to(DEVICE), masks.to(DEVICE)
            
            # Power embedding
            pe = F.normalize(model(signals, stats), p=2, dim=1)  # [B, 384]
            
            # Similarity to all attributes
            sim = (pe @ attr_embs.T) / TEMPERATURE  # [B, K]
            
            # Multi-positive loss
            # Maximize similarity to positive attributes, minimize to negative
            pos_mask = masks  # [B, K]
            neg_mask = 1 - masks
            
            # For each sample: -log(sum(exp(pos)) / sum(exp(all)))
            exp_sim = torch.exp(sim)
            pos_sum = (exp_sim * pos_mask).sum(dim=1)
            all_sum = exp_sim.sum(dim=1)
            loss = -torch.log(pos_sum / (all_sum + 1e-8)).mean()
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(loader)
        history.append(avg_loss)
        
        if (ep+1) % 5 == 0 or ep == 0:
            print(f'Epoch {ep+1}/{epochs} | Loss: {avg_loss:.4f}')
    
    torch.save(model.state_dict(), 'attr_power_clip_vnext.pth')
    print('✅ Model saved: attr_power_clip_vnext.pth')
    
    return model, text_model, attr_embs, history

model, text_model, attr_embs, history = train_attribute_model(train_ds)

In [ ]:
# @title 📈 Training Curve
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(range(1, len(history)+1), history, 'b-o', lw=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Attribute-Level Contrastive Training')
plt.grid(True, alpha=0.3)
plt.savefig('attr_training_curve.png', dpi=150)
plt.show()

In [ ]:
# @title 🎯 Inference: Attribute Activation + Device Reasoning

def infer_device(model, text_model, attr_embs, signal, stats):
    """Power → Attributes → Device"""
    model.eval()
    
    with torch.no_grad():
        signal_t = torch.tensor(signal).unsqueeze(0).to(DEVICE)
        stats_t = torch.tensor(stats).unsqueeze(0).to(DEVICE)
        
        # Power embedding
        pe = F.normalize(model(signal_t, stats_t), p=2, dim=1)
        
        # Attribute similarities
        sim = (pe @ attr_embs.T).squeeze().cpu().numpy()  # [K]
    
    # Soft activation
    activations = 1 / (1 + np.exp(-sim * 5))  # sigmoid with scaling
    
    # Device reasoning
    device_scores = {}
    for device, expected_attrs in DEVICE_ATTRS.items():
        indices = [ATTR_IDS.index(a) for a in expected_attrs]
        score = np.mean([activations[i] for i in indices])
        device_scores[device] = score
    
    # Top attributes
    top_attrs = [(ATTR_IDS[i], ATTRIBUTES[ATTR_IDS[i]]['name'], activations[i]) 
                 for i in np.argsort(-activations)[:5]]
    
    # Top devices
    sorted_devices = sorted(device_scores.items(), key=lambda x: -x[1])
    
    return {
        'activations': dict(zip(ATTR_IDS, activations)),
        'top_attrs': top_attrs,
        'device_scores': device_scores,
        'sorted_devices': sorted_devices
    }

In [ ]:
# @title 📊 Evaluation: Attribute & Device Metrics

def evaluate_model(model, text_model, attr_embs):
    # Load test samples (use full dataset for now)
    test_samples = []
    ws = 60
    with open('combination_labels.jsonl') as f:
        lines = f.readlines()
    
    for i in range(0, min(10000, len(lines)-ws), ws):
        chunk = [json.loads(lines[j]) for j in range(i, i+ws)]
        act = set(chunk[ws//2]['active_appliances']) & set(ALL_DEVICES)
        if len(act) != 1: continue  # Pure single device
        
        signal = np.array([c['aggregate_power'] for c in chunk], dtype=np.float32)
        device = list(act)[0]
        test_samples.append({'signal': signal, 'stats': compute_stats(signal), 'device': device})
    
    print(f'📊 Test samples: {len(test_samples)}')
    
    # Evaluate
    device_ranks = []
    attr_results = {a: [] for a in ATTR_IDS}
    
    for sample in tqdm(test_samples, desc='Evaluating'):
        result = infer_device(model, text_model, attr_embs, sample['signal'], sample['stats'])
        gt_device = sample['device']
        gt_attrs = set(DEVICE_ATTRS[gt_device])
        
        # Device rank
        sorted_devs = [d[0] for d in result['sorted_devices']]
        rank = sorted_devs.index(gt_device) + 1
        device_ranks.append(rank)
        
        # Attribute correctness
        for attr_id in ATTR_IDS:
            is_positive = attr_id in gt_attrs
            activation = result['activations'][attr_id]
            attr_results[attr_id].append((is_positive, activation))
    
    # Device metrics
    avg_rank = np.mean(device_ranks)
    top1 = np.mean([r == 1 for r in device_ranks]) * 100
    top3 = np.mean([r <= 3 for r in device_ranks]) * 100
    
    print('\n📊 Device-Level Results:')
    print(f'   Avg Rank: {avg_rank:.2f} / 6 (random = 3.5)')
    print(f'   Top-1: {top1:.1f}% (random = 16.7%)')
    print(f'   Top-3: {top3:.1f}% (random = 50%)')
    
    return {
        'device_ranks': device_ranks,
        'avg_rank': avg_rank,
        'top1': top1,
        'top3': top3,
        'attr_results': attr_results
    }

eval_results = evaluate_model(model, text_model, attr_embs)

In [ ]:
# @title 🤖 LLM Explanation (Gemini)
import google.generativeai as genai

try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    llm = genai.GenerativeModel('gemini-2.0-flash-exp')
    print('✅ Gemini connected')
except:
    llm = None
    print('⚠️ Gemini not configured')

def llm_explain(result, gt_device=None):
    top_attrs = result['top_attrs']
    sorted_devs = result['sorted_devices'][:3]
    
    prompt = f"""你是一位電力系統專家。根據以下功率信號的屬性分析，解釋設備推論結果。

## 檢測到的屬性
{chr(10).join([f"- {a[1]}: {a[2]:.2f}" for a in top_attrs])}

## 設備推論結果
{chr(10).join([f"{i+1}. {d[0]}: {d[1]:.2f}" for i, d in enumerate(sorted_devs)])}

{f'## 實際設備: {gt_device}' if gt_device else ''}

請簡要解釋：
1. 為什麼 Top-1 設備最可能？
2. 這些屬性如何對應到設備的物理特性？

請用繁體中文回答，100字內。"""
    
    if llm:
        try:
            response = llm.generate_content(prompt)
            return response.text
        except Exception as e:
            return f'Error: {e}'
    else:
        return '【LLM 未配置】'

In [ ]:
# @title 🔬 Demo: Single Sample Analysis

# Pick a random test sample
ws = 60
with open('combination_labels.jsonl') as f:
    lines = f.readlines()

# Find a fridge sample
for i in range(1000, len(lines)-ws, ws):
    chunk = [json.loads(lines[j]) for j in range(i, i+ws)]
    act = set(chunk[ws//2]['active_appliances']) & set(ALL_DEVICES)
    if 'fridge' in act and len(act) == 1:
        signal = np.array([c['aggregate_power'] for c in chunk], dtype=np.float32)
        stats = compute_stats(signal)
        break

result = infer_device(model, text_model, attr_embs, signal, stats)

print('📊 Sample: Fridge')
print(f'\n🔝 Top-5 Attributes:')
for attr_id, name, score in result['top_attrs']:
    print(f'   {attr_id} ({name}): {score:.3f}')

print(f'\n🏠 Device Scores:')
for device, score in result['sorted_devices']:
    marker = '✅' if device == 'fridge' else '  '
    print(f'   {marker} {device}: {score:.3f}')

print(f'\n🤖 LLM Explanation:')
print(llm_explain(result, 'fridge'))

In [ ]:
# @title 💾 Save Results

with open('vnext_attr_results.json', 'w') as f:
    json.dump({
        'experiment': 'Power-CLIP vNext Attribute-Level',
        'epochs': EPOCHS,
        'n_attributes': len(ATTR_IDS),
        'device_metrics': {
            'avg_rank': eval_results['avg_rank'],
            'top1': eval_results['top1'],
            'top3': eval_results['top3']
        },
        'loss_history': history
    }, f, indent=2)

from google.colab import files
for fn in ['vnext_attr_results.json', 'attr_training_curve.png', 'attr_power_clip_vnext.pth']:
    if os.path.exists(fn):
        print(f'⬇️ {fn}')
        try: files.download(fn)
        except: pass